### Monash Data Analytics Bootcamp - Final Project Assignment 

#### Project Team Members:
*   Megan Greenhill 
*   Hesh Kuruppuge
*   Jacqueline Xia
*   Mike Murphy 

The John Hopkins University (JHU) Covid data extract component of this assignment is based on the tutorial by B Chen 
and is used with his permission. The tutorial is located at the following link. 

*    https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6 
    

This assignment uses the following Covid19 CSV files which are updated daily and published by John Hopkins University. 

*    time_series_covid19_confirmed_global.csv................confirmed global Covid_19 cases
*    time_series_covid19_deaths_global.csv...................confirmed global Covid_19 deaths
*    time_series_covid19_recovered_global.csv................confirmed global Covid_19 recovered 

It uses the following World Covid Vaccination dataset published by Our World in Data. 

*    https://ourworldindata.org/covid-vaccinations 

It uses the following World Population dataset published by Our World in Data. 

*    https://www.worldometers.info/world-population/population-by-country/ 



In [1]:
#   import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
from urllib.error import HTTPError
import numpy as np
import wget
import time
from datetime import datetime
from sqlalchemy import create_engine
# from pandas_profiling import ProfileReport

#   the project uses urls / wget downloads in place of API calls are they are not available for the datasets needed

# url of the raw csv dataset
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]
[wget.download(url) for url in urls]

['time_series_covid19_confirmed_global (4).csv',
 'time_series_covid19_deaths_global (4).csv',
 'time_series_covid19_recovered_global (4).csv']

In [2]:
#   create dataframes from csv's

confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
    
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
    
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

confirmed_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/19/22,4/20/22,4/21/22,4/22/22,4/23/22,4/24/22,4/25/22,4/26/22,4/27/22,4/28/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,178513,178574,178611,178638,178648,178689,178745,178769,178809,178850
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,274535,274606,274606,274737,274791,274828,274828,274862,274929,275002
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265746,265746,265754,265761,265761,265767,265771,265772,265773,265776
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,40709,41013,41013,41013,41013,41013,41013,41013,41013,41349
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,99287,99287,99287,99287,99287,99287,99287,99287,99287,99287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,656876,656876,656876,656876,656876,656876,656876,657060,657060,657060
280,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
281,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11818,11818,11818,11818,11818,11818,11818,11818,11818,11818
282,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,318640,318640,318984,318984,318984,318984,319316,319431,319431,319594


In [3]:
deaths_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/19/22,4/20/22,4/21/22,4/22/22,4/23/22,4/24/22,4/25/22,4/26/22,4/27/22,4/28/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,7680,7680,7680,7680,7681,7682,7683,7683,7683,7683
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,3496,3496,3496,3496,3496,3496,3496,3496,3496,3496
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,6874,6874,6874,6874,6874,6875,6875,6875,6875,6875
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,153,153,153,153,153,153,153,153,153,153
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,5657,5657,5657,5657,5657,5657,5657,5657,5657,5657
280,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,2148,2148,2148,2148,2148,2149,2149,2149,2149,2149
282,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,3973,3973,3974,3974,3974,3974,3976,3976,3976,3976


In [4]:
recovered_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/19/22,4/20/22,4/21/22,4/22/22,4/23/22,4/24/22,4/25/22,4/26/22,4/27/22,4/28/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#   save DFs to CSVs to do exploratory data analysis
confirmed_df.to_csv('exported_conf_df.csv')
deaths_df.to_csv('exported_deaths_df.csv')
recovered_df.to_csv('exported_recvd_df.csv')

In [6]:
#   use melt() to unpivot DataFrames from current wide format into long format
#   NOTE: Included 'dates = name_of__df.columns[4:]' for each of the CSVs to ensure that they all work if the CSVs
#   have differing numbers of columns

dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
dates = deaths_df.columns[4:]
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
dates = recovered_df.columns[4:]
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)
confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
235147,NaN,West Bank and Gaza,31.952200,35.233200,4/28/22,657060
235148,NaN,Winter Olympics 2022,39.904200,116.407400,4/28/22,535
235149,NaN,Yemen,15.552727,48.516388,4/28/22,11818
235150,NaN,Zambia,-13.133897,27.849332,4/28/22,319594


In [7]:
recovered_df_long

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
222727,NaN,West Bank and Gaza,31.952200,35.233200,4/28/22,0
222728,NaN,Winter Olympics 2022,39.904200,116.407400,4/28/22,0
222729,NaN,Yemen,15.552727,48.516388,4/28/22,0
222730,NaN,Zambia,-13.133897,27.849332,4/28/22,0


#### Remove recovered data for Canada due to mismatch issue
*    Canada recovered data is counted for the whole Country instead of by Province/State which is how Canada 
*        and the rest of the world count data for "Confirmed Cases" and "Deaths".

##### We considered apportioning recovered data for the whole country in the same ratio as confirmed cases. 
##### This is arbitrarily altering data from a source and is considered bad practice so we did not do so. 


In [8]:
#   remove Canada recovered data
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

####   merge the 3 data frames one after another

In [9]:
#   merge confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

#    merge full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
235147,NaN,West Bank and Gaza,31.952200,35.233200,4/28/22,657060,5657,0.0
235148,NaN,Winter Olympics 2022,39.904200,116.407400,4/28/22,535,0,0.0
235149,NaN,Yemen,15.552727,48.516388,4/28/22,11818,2149,0.0
235150,NaN,Zambia,-13.133897,27.849332,4/28/22,319594,3976,0.0


In [10]:
#   Check Canada data in "full_table" - "recovered" should be 0 and check of CSV file confirms it is 
full_table.to_csv('full_table_can_recvd.csv')

####   Data Cleansing
*    1. converting date from string to datetime
*    2. replacing missing value NaN with zeroes
*    3. coronavirus cases reported from 3 cruise ships should be treated differently

In [11]:
#   1 converting date from string to datetime
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [12]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
235147,NaN,West Bank and Gaza,31.952200,35.233200,2022-04-28,657060,5657,0.0
235148,NaN,Winter Olympics 2022,39.904200,116.407400,2022-04-28,535,0,0.0
235149,NaN,Yemen,15.552727,48.516388,2022-04-28,11818,2149,0.0
235150,NaN,Zambia,-13.133897,27.849332,2022-04-28,319594,3976,0.0


In [13]:
#   2 replacing missing values NaN
#   detect missing values NaN 
full_table.isna().sum()

Province/State    161460
Country/Region         0
Lat                 1656
Long                1656
Date                   0
Confirmed              0
Deaths                 0
Recovered          17388
dtype: int64

In [14]:
#   replace 'recovered' NaNs with zero
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Recovered']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
235147    0.0
235148    0.0
235149    0.0
235150    0.0
235151    0.0
Name: Recovered, Length: 235152, dtype: float64

####   3 coronavirus cases reported from 3 cruise ships should be treated differently
*    Cases reported from cruise ships: Grand Princess, Diamond Princess and MS Zaandam need to be extracted and 
*    treated differently due to Province/State and Country/Region mismatch over time

In [15]:
#   extract data for these ships 
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

#   remove data for these ships 
full_table = full_table[~(ship_rows)]

In [16]:
#   check "full_table" to ensure its integrity
full_table.to_csv('full_table_clean.csv')

#### Calculate the number of Active Cases 
    *Active Cases = Confirmed Cases - Deaths - Recovered Cases 
    

In [17]:
#    calculate active cases = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']
full_table

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/2074956377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
235147,NaN,West Bank and Gaza,31.952200,35.233200,2022-04-28,657060,5657,0.0,651403.0
235148,NaN,Winter Olympics 2022,39.904200,116.407400,2022-04-28,535,0,0.0,535.0
235149,NaN,Yemen,15.552727,48.516388,2022-04-28,11818,2149,0.0,9669.0
235150,NaN,Zambia,-13.133897,27.849332,2022-04-28,319594,3976,0.0,315618.0


In [18]:
#   aggregate data into Country/Region and group by Date and Country/Region
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
full_grouped

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/939450847.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
163111,2022-04-28,West Bank and Gaza,657060,5657,0.0,651403.0
163112,2022-04-28,Winter Olympics 2022,535,0,0.0,535.0
163113,2022-04-28,Yemen,11818,2149,0.0,9669.0
163114,2022-04-28,Zambia,319594,3976,0.0,315618.0


In [19]:
#   calculate daily New cases, New deaths and New recovered by deducting the corresponding accumulative data on the previous day
#   new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

#   renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

#   merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

#   filling na with 0
full_grouped = full_grouped.fillna(0)

#   fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/3349694554.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


In [20]:
#   Adding column for calculating case fatality, and replacing any NaN values with 0
full_grouped["Case Fatality"] = round((full_grouped["Deaths"] / full_grouped["Confirmed"] * 100), 2)
full_grouped["Case Fatality"] = full_grouped["Case Fatality"].fillna(0)

####   Final output is data sorted by Date and Country/Region ascending where: -
*    Confirmed Cases, Deaths, Recovered and Active are cumulative data for the entire period, and,
*    New cases, New deaths and New Recovered are daily data

In [21]:
#    Final data frame
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Case Fatality
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0,0.00
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0,0.00
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0,0.00
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0,0.00
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...
163111,2022-04-28,West Bank and Gaza,657060,5657,0.0,651403.0,0,0,0,0.86
163112,2022-04-28,Winter Olympics 2022,535,0,0.0,535.0,0,0,0,0.00
163113,2022-04-28,Yemen,11818,2149,0.0,9669.0,0,0,0,18.18
163114,2022-04-28,Zambia,319594,3976,0.0,315618.0,163,0,0,1.24


#### Check data for Australia against JHU Daily Replorts to confirm that it is correct 
#### Data is correct as at 07/02/2022 - 2,727,260 confirmed cases & 4,200 confirmed deaths


In [22]:
#   select Oz to check that data is correct
full_grouped_oz = full_grouped.loc[full_grouped["Country/Region"] == "Australia"]
full_grouped_oz

# Check data for Australia against JHU Daily Replorts to confirm that it is correct. 
# Data is correct as at 23/04/2022 - 5,685,602 confirmed cases & 6,983 confirmed deaths

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Case Fatality
9,2020-01-22,Australia,0,0,0.0,0.0,0,0,0,0.00
206,2020-01-23,Australia,0,0,0.0,0.0,0,0,0,0.00
403,2020-01-24,Australia,0,0,0.0,0.0,0,0,0,0.00
600,2020-01-25,Australia,0,0,0.0,0.0,0,0,0,0.00
797,2020-01-26,Australia,4,0,0.0,4.0,4,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...
162140,2022-04-24,Australia,5719311,7006,0.0,5712305.0,33709,23,0,0.12
162337,2022-04-25,Australia,5756791,7052,0.0,5749739.0,37480,46,0,0.12
162534,2022-04-26,Australia,5796700,7112,0.0,5789588.0,39909,60,0,0.12
162731,2022-04-27,Australia,5847576,7164,0.0,5840412.0,50876,52,0,0.12


#### VACCINATION DATASET SOURCED FROM OUR WORLD IN DATA AT THE FOLLOWING LINK
*    https://ourworldindata.org/covid-vaccinations


In [23]:
#   read the vaccination dataset - csv file into a dataframe
vacc_data_df = pd.read_csv('owid_vaccinations.csv')
vacc_data_df

C:\Users\mikem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,24/2/20,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,25/2/20,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,26/2/20,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,27/2/20,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,28/2/20,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180351,ZWE,Africa,Zimbabwe,15/4/22,247237.0,29.0,44.571,5462.0,0.0,0.714,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
180352,ZWE,Africa,Zimbabwe,16/4/22,247237.0,0.0,44.571,5462.0,0.0,0.714,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
180353,ZWE,Africa,Zimbabwe,17/4/22,247278.0,41.0,45.714,5463.0,1.0,0.857,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
180354,ZWE,Africa,Zimbabwe,18/4/22,247297.0,19.0,41.000,5464.0,1.0,0.571,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


#### Create data frame with just people vaccinated data 


In [24]:
vacc_data_df = vacc_data_df[["iso_code", "continent", "location", "date", "people_vaccinated_per_hundred",
                             "people_fully_vaccinated_per_hundred", "total_boosters_per_hundred"]]
vacc_data_df

,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,AFG,Asia,Afghanistan,24/2/20,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,25/2/20,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,26/2/20,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,27/2/20,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,28/2/20,NaN,NaN,NaN
...,...,...,...,...,...,...,...
180351,ZWE,Africa,Zimbabwe,15/4/22,38.27,23.96,4.85
180352,ZWE,Africa,Zimbabwe,16/4/22,38.35,23.96,4.86
180353,ZWE,Africa,Zimbabwe,17/4/22,38.39,23.97,4.87
180354,ZWE,Africa,Zimbabwe,18/4/22,38.42,23.97,4.88


In [25]:
#   detect missing values NaN 
vacc_data_df.isna().sum()

iso_code                                    0
continent                               10552
location                                    0
date                                        0
people_vaccinated_per_hundred          133796
people_fully_vaccinated_per_hundred    136344
total_boosters_per_hundred             158758
dtype: int64

In [26]:
#   replace NaNs with zero
vacc_data_df['people_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
vacc_data_df['people_fully_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
vacc_data_df['total_boosters_per_hundred'] = vacc_data_df['total_boosters_per_hundred'].fillna(0)
vacc_data_df

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/3241861704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/3241861704.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_fully_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/3241861704.py:4: Setti

,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,AFG,Asia,Afghanistan,24/2/20,0.00,0.00,0.00
1,AFG,Asia,Afghanistan,25/2/20,0.00,0.00,0.00
2,AFG,Asia,Afghanistan,26/2/20,0.00,0.00,0.00
3,AFG,Asia,Afghanistan,27/2/20,0.00,0.00,0.00
4,AFG,Asia,Afghanistan,28/2/20,0.00,0.00,0.00
...,...,...,...,...,...,...,...
180351,ZWE,Africa,Zimbabwe,15/4/22,38.27,38.27,4.85
180352,ZWE,Africa,Zimbabwe,16/4/22,38.35,38.35,4.86
180353,ZWE,Africa,Zimbabwe,17/4/22,38.39,38.39,4.87
180354,ZWE,Africa,Zimbabwe,18/4/22,38.42,38.42,4.88


In [27]:
#   data cleansing'Country/Region].replace(['United States'], 'US')
vacc_data_df['location'] = vacc_data_df['location'].replace(['United States'],'US')
vacc_data_df

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/797748420.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['location'] = vacc_data_df['location'].replace(['United States'],'US')


,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,AFG,Asia,Afghanistan,24/2/20,0.00,0.00,0.00
1,AFG,Asia,Afghanistan,25/2/20,0.00,0.00,0.00
2,AFG,Asia,Afghanistan,26/2/20,0.00,0.00,0.00
3,AFG,Asia,Afghanistan,27/2/20,0.00,0.00,0.00
4,AFG,Asia,Afghanistan,28/2/20,0.00,0.00,0.00
...,...,...,...,...,...,...,...
180351,ZWE,Africa,Zimbabwe,15/4/22,38.27,38.27,4.85
180352,ZWE,Africa,Zimbabwe,16/4/22,38.35,38.35,4.86
180353,ZWE,Africa,Zimbabwe,17/4/22,38.39,38.39,4.87
180354,ZWE,Africa,Zimbabwe,18/4/22,38.42,38.42,4.88


#### people_vaccinated_per_hundred is non zero if people have had one vaccination, so we can calculate the 
#### unvaccinated as follows: people_not_vaccinated_per_hundred = 100 - people_vaccinated_per_hundred.


In [28]:
#   now calculate people_not_vaccinated_per_hundred = 100 - people_vaccinated_per_hundred 
vacc_data_df['people_not_vaccinated_per_hundred'] = 0
vacc_data_df['people_not_vaccinated_per_hundred'] = np.where(vacc_data_df['people_vaccinated_per_hundred'] != 0,
                        100 - vacc_data_df['people_vaccinated_per_hundred'], vacc_data_df['people_not_vaccinated_per_hundred'])

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/1020831299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_not_vaccinated_per_hundred'] = 0
C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/1020831299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_not_vaccinated_per_hundred'] = np.where(vacc_data_df['people_vaccinated_per_hundred'] != 0,


In [29]:
vacc_data_df

,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,people_not_vaccinated_per_hundred
0,AFG,Asia,Afghanistan,24/2/20,0.00,0.00,0.00,0.00
1,AFG,Asia,Afghanistan,25/2/20,0.00,0.00,0.00,0.00
2,AFG,Asia,Afghanistan,26/2/20,0.00,0.00,0.00,0.00
3,AFG,Asia,Afghanistan,27/2/20,0.00,0.00,0.00,0.00
4,AFG,Asia,Afghanistan,28/2/20,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...
180351,ZWE,Africa,Zimbabwe,15/4/22,38.27,38.27,4.85,61.73
180352,ZWE,Africa,Zimbabwe,16/4/22,38.35,38.35,4.86,61.65
180353,ZWE,Africa,Zimbabwe,17/4/22,38.39,38.39,4.87,61.61
180354,ZWE,Africa,Zimbabwe,18/4/22,38.42,38.42,4.88,61.58


#### WORLD POPULATION DATASET AS AT 31/12/2020 SOURCED FROM THE FOLLOWING LINK
*    https://www.worldometers.info/world-population/population-by-country/ 

In [30]:
#   read the world population dataset - csv file into a dataframe
pop_data_df = pd.read_csv('world_populations.csv')
pop_data_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102050.0,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,532760424.0,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,30117411.0,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,360285439.0,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,24220660.0,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1775378.0
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,...,23807586.0,24473176.0,25147112.0,25823488.0,26497881.0,27168210.0,27834811.0,28498683.0,29161922.0,29825968.0
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,...,52003759.0,52832659.0,53687125.0,54544184.0,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,...,14023199.0,14465148.0,14926551.0,15399793.0,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0


In [31]:
#   detect missing values NaN 
pop_data_df.isna().sum()

Country Name      0
Country Code      0
Indicator Name    0
Indicator Code    0
1960              2
                 ..
2016              1
2017              1
2018              1
2019              1
2020              1
Length: 65, dtype: int64

In [32]:
#   replace NaNs with zero
pop_data_df['1960'] = pop_data_df['1960'].fillna(0)
pop_data_df['2016'] = pop_data_df['2016'].fillna(0)
pop_data_df['2017'] = pop_data_df['2017'].fillna(0)
pop_data_df['2018'] = pop_data_df['2018'].fillna(0)
pop_data_df['2019'] = pop_data_df['2019'].fillna(0)
pop_data_df['2020'] = pop_data_df['2020'].fillna(0)
pop_data_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102050.0,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,532760424.0,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,30117411.0,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,360285439.0,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,24220660.0,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1775378.0
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,...,23807586.0,24473176.0,25147112.0,25823488.0,26497881.0,27168210.0,27834811.0,28498683.0,29161922.0,29825968.0
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,...,52003759.0,52832659.0,53687125.0,54544184.0,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,...,14023199.0,14465148.0,14926551.0,15399793.0,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0


#### Create dataframe for Country Codes table

In [33]:
# Copy OWID Vaccination dataframe, as we want to use OWID country codes
country_codes = vacc_data_df[["iso_code","location","continent"]]
country_codes_cleaned = country_codes.drop_duplicates(subset=["iso_code"])
country_codes = country_codes_cleaned.rename(columns={"iso_code":"country_id",
                                              "location":"country_name",
                                              "continent":"continent_name"})

In [34]:
#Adding countries to country code dataframe to match population dataframe
africa_eastern = {'country_id': 'AFE', 'country_name': 'Africa Eastern and Southern', 'continent_name': 'Africa'}
africa_western = {'country_id': 'AFW', 'country_name': 'Africa Western and Central', 'continent_name': 'Africa'}
arab_world = {'country_id': 'ARB', 'country_name': 'Arab World', 'continent_name': 'Asia'}
asm = {'country_id': 'ASM', 'country_name': 'American Samoa', 'continent_name': 'Oceania'}
chi = {'country_id': 'CHI', 'country_name': 'Channel Islands', 'continent_name': 'Europe'}
prk = {'country_id': 'PRK', 'country_name': 'Korea, Dem. Peoples Rep.', 'continent_name': 'Asia'}
xkx = {'country_id': 'XKX', 'country_name': 'Kosovo', 'continent_name': 'Europe'}
maf = {'country_id': 'MAF', 'country_name': 'St. Martin', 'continent_name': 'North America'}

country_codes = country_codes.append(africa_eastern, ignore_index = True)
country_codes = country_codes.append(africa_western, ignore_index = True)
country_codes = country_codes.append(arab_world, ignore_index = True)
country_codes = country_codes.append(asm, ignore_index = True)
country_codes = country_codes.append(chi, ignore_index = True)
country_codes = country_codes.append(prk, ignore_index = True)
country_codes = country_codes.append(xkx, ignore_index = True)
country_codes = country_codes.append(maf, ignore_index = True)

country_codes

,country_id,country_name,continent_name
0,AFG,Afghanistan,Asia
1,OWID_AFR,Africa,NaN
2,ALB,Albania,Europe
3,DZA,Algeria,Africa
4,AND,Andorra,Europe
...,...,...,...
245,ASM,American Samoa,Oceania
246,CHI,Channel Islands,Europe
247,PRK,"Korea, Dem. Peoples Rep.",Asia
248,XKX,Kosovo,Europe


#### Edit full_grouped covid case dataframe to include country ID

In [35]:
full_grouped_copy = full_grouped.copy()
full_grouped_copy = full_grouped_copy.rename(columns={"Country/Region":"country_name"})
full_grouped_reformatted = pd.merge(country_codes,full_grouped_copy,on="country_name")

### Change structure of dataframes to match structure of tables created in the database

In [36]:
# Covid Cases table
# Copy only the columns needed into a new DataFrame.
covid_cases = full_grouped_reformatted[["country_id","Date","Confirmed","Deaths","Recovered","Active","Case Fatality","New cases","New deaths","New recovered"]]

# Rename columns to fit the tables created in the database.
covid_cases = covid_cases.rename(columns={"Date":"date",
                                          "Confirmed":"confirmed",
                                          "Deaths":"deaths",
                                          "Recovered":"recovered",
                                          "Active":"active",
                                          "Case Fatality":"case_fatality",
                                          "New cases":"new_cases",
                                          "New deaths":"new_deaths",
                                          "New recovered":"new_recovered"})
covid_cases

,country_id,date,confirmed,deaths,recovered,active,case_fatality,new_cases,new_deaths,new_recovered
0,AFG,2020-01-22,0,0,0.0,0.0,0.00,0,0,0
1,AFG,2020-01-23,0,0,0.0,0.0,0.00,0,0,0
2,AFG,2020-01-24,0,0,0.0,0.0,0.00,0,0,0
3,AFG,2020-01-25,0,0,0.0,0.0,0.00,0,0,0
4,AFG,2020-01-26,0,0,0.0,0.0,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...
152347,ZWE,2022-04-24,247524,5468,0.0,242056.0,2.21,0,0,0
152348,ZWE,2022-04-25,247598,5468,0.0,242130.0,2.21,74,0,0
152349,ZWE,2022-04-26,247667,5469,0.0,242198.0,2.21,69,1,0
152350,ZWE,2022-04-27,247729,5469,0.0,242260.0,2.21,62,0,0


In [37]:
# Population table
# Copy only the columns needed into a new DataFrame.
population = pop_data_df[["Country Code","2020"]]

# Remove country codes that aren't actually countries
population = population[population["Country Code"].str.contains('CEB')==False]
population = population[population["Country Code"].str.contains('Country Code')==False]
population = population[population["Country Code"].str.contains('CSS')==False]
population = population[population["Country Code"].str.contains('EAP')==False]
population = population[population["Country Code"].str.contains('EAR')==False]
population = population[population["Country Code"].str.contains('EAS')==False]
population = population[population["Country Code"].str.contains('ECA')==False]
population = population[population["Country Code"].str.contains('ECS')==False]
population = population[population["Country Code"].str.contains('EMU')==False]
population = population[population["Country Code"].str.contains('EUU')==False]
population = population[population["Country Code"].str.contains('FCS')==False]
population = population[population["Country Code"].str.contains('HIC')==False]
population = population[population["Country Code"].str.contains('HPC')==False]
population = population[population["Country Code"].str.contains('IBD')==False]
population = population[population["Country Code"].str.contains('IBT')==False]
population = population[population["Country Code"].str.contains('IDA')==False]
population = population[population["Country Code"].str.contains('IDB')==False]
population = population[population["Country Code"].str.contains('IDX')==False]
population = population[population["Country Code"].str.contains('INX')==False]
population = population[population["Country Code"].str.contains('LAC')==False]
population = population[population["Country Code"].str.contains('LCN')==False]
population = population[population["Country Code"].str.contains('LDC')==False]
population = population[population["Country Code"].str.contains('LIC')==False]
population = population[population["Country Code"].str.contains('LMC')==False]
population = population[population["Country Code"].str.contains('LMY')==False]
population = population[population["Country Code"].str.contains('LTE')==False]
population = population[population["Country Code"].str.contains('MEA')==False]
population = population[population["Country Code"].str.contains('MIC')==False]
population = population[population["Country Code"].str.contains('MNA')==False]
population = population[population["Country Code"].str.contains('NAC')==False]
population = population[population["Country Code"].str.contains('OED')==False]
population = population[population["Country Code"].str.contains('OSS')==False]
population = population[population["Country Code"].str.contains('PRE')==False]
population = population[population["Country Code"].str.contains('PSS')==False]
population = population[population["Country Code"].str.contains('PST')==False]
population = population[population["Country Code"].str.contains('SAS')==False]
population = population[population["Country Code"].str.contains('SSA')==False]
population = population[population["Country Code"].str.contains('SSF')==False]
population = population[population["Country Code"].str.contains('SST')==False]
population = population[population["Country Code"].str.contains('TEA')==False]
population = population[population["Country Code"].str.contains('TEC')==False]
population = population[population["Country Code"].str.contains('TLA')==False]
population = population[population["Country Code"].str.contains('TMN')==False]
population = population[population["Country Code"].str.contains('TSA')==False]
population = population[population["Country Code"].str.contains('TSS')==False]
population = population[population["Country Code"].str.contains('UMC')==False]
population = population[population["Country Code"].str.contains('WLD')==False]

# Rename columns to fit the tables created in the database.
population = population.rename(columns={"Country Code":"country_id",
                                          "2020":"population"})
population

,country_id,population
0,ABW,106766.0
1,AFE,677243299.0
2,AFG,38928341.0
3,AFW,458803476.0
4,AGO,32866268.0
...,...,...
261,XKX,1775378.0
262,YEM,29825968.0
263,ZAF,59308690.0
264,ZMB,18383956.0


In [38]:
# Vaccinations table
# Copy only the columns needed into a new DataFrame.
vaccinations = vacc_data_df[["iso_code","date","people_vaccinated_per_hundred", "people_fully_vaccinated_per_hundred",
                             "total_boosters_per_hundred", "people_not_vaccinated_per_hundred"]]

# Rename columns to fit the tables created in the database.
vaccinations = vaccinations.rename(columns={"iso_code":"country_id",
                                          "people_vaccinated_per_hundred":"vaccinated_per_hundred",
                                          "people_fully_vaccinated_per_hundred":"fully_vaccinated_per_hundred",
                                          "total_boosters_per_hundred":"boosted_per_hundred",
                                          "people_not_vaccinated_per_hundred":"not_fully_vaccinated_per_hundred"})
vaccinations

,country_id,date,vaccinated_per_hundred,fully_vaccinated_per_hundred,boosted_per_hundred,not_fully_vaccinated_per_hundred
0,AFG,24/2/20,0.00,0.00,0.00,0.00
1,AFG,25/2/20,0.00,0.00,0.00,0.00
2,AFG,26/2/20,0.00,0.00,0.00,0.00
3,AFG,27/2/20,0.00,0.00,0.00,0.00
4,AFG,28/2/20,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...
180351,ZWE,15/4/22,38.27,38.27,4.85,61.73
180352,ZWE,16/4/22,38.35,38.35,4.86,61.65
180353,ZWE,17/4/22,38.39,38.39,4.87,61.61
180354,ZWE,18/4/22,38.42,38.42,4.88,61.58


In [39]:
# Change data type of vaccination dates to datetime
vaccinations["date"] = pd.to_datetime(vaccinations["date"])
covid_cases["date"] = pd.to_datetime(covid_cases["date"])

In [40]:
# Create full table with merged information
full_covid_table_1 = pd.merge(covid_cases, vaccinations,  how='outer', on=['country_id','date'])
full_covid_table_2 = pd.merge(full_covid_table_1, country_codes, how='outer', on = 'country_id')
full_covid_table_3 = pd.merge(full_covid_table_2, population, how='outer', on = 'country_id')
full_covid_table = full_covid_table_3[["country_id","country_name","continent_name","population","date","confirmed","deaths","recovered","active","case_fatality","new_cases","new_deaths","new_recovered","vaccinated_per_hundred","fully_vaccinated_per_hundred","boosted_per_hundred","not_fully_vaccinated_per_hundred"]]
full_covid_table = full_covid_table.fillna(0)
full_covid_table

,country_id,country_name,continent_name,population,date,confirmed,deaths,recovered,active,case_fatality,new_cases,new_deaths,new_recovered,vaccinated_per_hundred,fully_vaccinated_per_hundred,boosted_per_hundred,not_fully_vaccinated_per_hundred
0,AFG,Afghanistan,Asia,38928341.0,2020-01-22 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFG,Afghanistan,Asia,38928341.0,2020-01-23 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AFG,Afghanistan,Asia,38928341.0,2020-01-24 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AFG,Afghanistan,Asia,38928341.0,2020-01-25 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AFG,Afghanistan,Asia,38928341.0,2020-01-26 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199898,ARB,Arab World,Asia,436080728.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199899,ASM,American Samoa,Oceania,55197.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199900,CHI,Channel Islands,Europe,173859.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199901,PRK,"Korea, Dem. Peoples Rep.",Asia,25778815.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# Create a DF for just those countries with hospitalisation data
filtered_df = full_covid_table[full_covid_table['country_id'].isin(["AUS","AUT","BEL","BOL","BGR","CAN","HRV","CYP","CZE","DNK","EST","FIN","FRA","HUN",
                                         "ISL","IRL","ISR","ITA","JPN","LVA","LTU","LUX","MYS","MLT","NLD","NOR","POL","PRT",
                                         "ROU","SRB","SVK","SVN","ZAF","ESP","SWE","CHE","GBR","USA"])]
filtered_df.dropna(axis=0, how="all")
filtered_df

,country_id,country_name,continent_name,population,date,confirmed,deaths,recovered,active,case_fatality,new_cases,new_deaths,new_recovered,vaccinated_per_hundred,fully_vaccinated_per_hundred,boosted_per_hundred,not_fully_vaccinated_per_hundred
6968,AUS,Australia,Oceania,25687041.0,2020-01-22 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
6969,AUS,Australia,Oceania,25687041.0,2020-01-23 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
6970,AUS,Australia,Oceania,25687041.0,2020-01-24 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
6971,AUS,Australia,Oceania,25687041.0,2020-01-25 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
6972,AUS,Australia,Oceania,25687041.0,2020-01-26 00:00:00,4.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152979,USA,US,North America,329484123.0,2022-08-04 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.25,77.25,29.80,22.75
152980,USA,US,North America,329484123.0,2022-09-04 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.26,77.26,29.82,22.74
152981,USA,US,North America,329484123.0,2022-10-04 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.27,77.27,29.83,22.73
152982,USA,US,North America,329484123.0,2022-11-04 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.29,77.29,29.86,22.71


In [42]:
# Save basic Covid cases for which Hospitalisation data is available
filtered_df.to_csv('../CSVs/Tables/covid_cases_hosp_data.csv')

In [43]:
# Convert 'date' to datetime format and change dateframe name for later merging with hospitalisations dataframe
filtered_df["date"] = pd.to_datetime(filtered_df["date"])
covid_case_df = filtered_df

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/1864857667.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["date"] = pd.to_datetime(filtered_df["date"])


In [44]:
# Get the Hospitalisation data and create dataframe
hosp_df = pd.read_csv('../CSVs/current-covid-patients-hospital.csv', 
                 parse_dates=['Day'])

In [45]:
# Drop unwanted column with Country name
hosp_df.drop('Entity', axis=1, inplace=True)
hosp_df

,Code,Day,Daily hospital occupancy
0,AUS,2020-03-31,426
1,AUS,2020-04-01,441
2,AUS,2020-04-02,415
3,AUS,2020-04-03,457
4,AUS,2020-04-04,490
...,...,...,...
25634,USA,2022-04-20,10917
25635,USA,2022-04-21,11093
25636,USA,2022-04-22,11196
25637,USA,2022-04-23,11131


In [46]:
# Change column names, and convert 'date' to datetime format
hosp_df.rename(columns={'Code' : 'country_id', 'Daily hospital occupancy':'hospital_occupancy', 'Day':'date'}, index={'ONE': 'Row_1'}, inplace=True)
hosp_df

,country_id,date,hospital_occupancy
0,AUS,2020-03-31,426
1,AUS,2020-04-01,441
2,AUS,2020-04-02,415
3,AUS,2020-04-03,457
4,AUS,2020-04-04,490
...,...,...,...
25634,USA,2022-04-20,10917
25635,USA,2022-04-21,11093
25636,USA,2022-04-22,11196
25637,USA,2022-04-23,11131


In [47]:
hosp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25639 entries, 0 to 25638
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   country_id          25639 non-null  object        
 1   date                25639 non-null  datetime64[ns]
 2   hospital_occupancy  25639 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 801.2+ KB


In [48]:
# merge basic Covid cases for which Hospitalisation data is available and hospitalisation data
full_covid_table=covid_case_df.merge(
  right=hosp_df, 
  how='left',
  on=['country_id', 'date']
)
full_covid_table

,country_id,country_name,continent_name,population,date,confirmed,deaths,recovered,active,case_fatality,new_cases,new_deaths,new_recovered,vaccinated_per_hundred,fully_vaccinated_per_hundred,boosted_per_hundred,not_fully_vaccinated_per_hundred,hospital_occupancy
0,AUS,Australia,Oceania,25687041.0,2020-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,NaN
1,AUS,Australia,Oceania,25687041.0,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,NaN
2,AUS,Australia,Oceania,25687041.0,2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,NaN
3,AUS,Australia,Oceania,25687041.0,2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,NaN
4,AUS,Australia,Oceania,25687041.0,2020-01-26,4.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.00,0.00,0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33061,USA,US,North America,329484123.0,2022-08-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.25,77.25,29.80,22.75,NaN
33062,USA,US,North America,329484123.0,2022-09-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.26,77.26,29.82,22.74,NaN
33063,USA,US,North America,329484123.0,2022-10-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.27,77.27,29.83,22.73,NaN
33064,USA,US,North America,329484123.0,2022-11-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.29,77.29,29.86,22.71,NaN


In [49]:
# Replace NaNs with zeroes
full_covid_table['hospital_occupancy'] = full_covid_table['hospital_occupancy'].fillna(0)
full_covid_table

,country_id,country_name,continent_name,population,date,confirmed,deaths,recovered,active,case_fatality,new_cases,new_deaths,new_recovered,vaccinated_per_hundred,fully_vaccinated_per_hundred,boosted_per_hundred,not_fully_vaccinated_per_hundred,hospital_occupancy
0,AUS,Australia,Oceania,25687041.0,2020-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
1,AUS,Australia,Oceania,25687041.0,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
2,AUS,Australia,Oceania,25687041.0,2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
3,AUS,Australia,Oceania,25687041.0,2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
4,AUS,Australia,Oceania,25687041.0,2020-01-26,4.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33061,USA,US,North America,329484123.0,2022-08-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.25,77.25,29.80,22.75,0.0
33062,USA,US,North America,329484123.0,2022-09-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.26,77.26,29.82,22.74,0.0
33063,USA,US,North America,329484123.0,2022-10-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.27,77.27,29.83,22.73,0.0
33064,USA,US,North America,329484123.0,2022-11-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.29,77.29,29.86,22.71,0.0


In [50]:
# Save all tables to CSVs for use in Javascript
country_codes.to_csv('../CSVs/Tables/country_codes.csv')
covid_cases.to_csv('../CSVs/Tables/covid_cases.csv')
population.to_csv('../CSVs/Tables/population.csv')
vaccinations.to_csv('../CSVs/Tables/vaccinations.csv')
full_covid_table.to_csv('../CSVs/Tables/full_covid_table.csv')

### Load tables to Integrated Covid View database

In [51]:
# Create database connection
rds_connection_string = "postgres:meg221196@localhost:5432/integrated_covid_view_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

# Confirm database tables
engine.table_names()

C:\Users\mikem\AppData\Local\Temp/ipykernel_19252/3867350856.py:6: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Load dataframes to database tables
full_covid_table.to_sql(name='full_covid_table', con=engine, if_exists='append', index=False)
country_codes.to_sql(name='country_codes', con=engine, if_exists='append', index=False)
covid_cases.to_sql(name='covid_cases', con=engine, if_exists='append', index=False)
population.to_sql(name='population', con=engine, if_exists='append', index=False)
vaccinations.to_sql(name='vaccinations', con=engine, if_exists='append', index=False)